In [46]:
import tifffile
import numpy as np
from PIL import Image
import pandas as pd

In [50]:
source_img_ffn = '/g/data1a/kl02/jss548/HailPixel/20181126_mosaic.tif'
output_path    = '/g/data1a/kl02/jss548/HailPixel/subsets'
output_ffn     = '/g/data1a/kl02/jss548/HailPixel/subsets/index.csv'

In [36]:
im    = tifffile.imread(source_img_ffn)
im_sz = np.shape(im)
print('original image size', im_sz)
im_y  = im_sz[0]
im_x  = im_sz[1]
im_z  = im_sz[2]


original image size (18509, 26418, 4)


In [37]:
# target size
subset_sz      = 500
subset_overlap = 50
# image size
im_pad_y = 18500
im_pad_x = 26500
# pad image
pad_y0 = 50
pad_x0 = 50
pad_yn = 50 + (im_pad_y-im_y)
pad_xn = 50 + (im_pad_x-im_x)
im_padded = np.pad(im, ((pad_y0, pad_yn), (pad_x0, pad_xn), (0, 0)), mode='constant')
print('padded image size', np.shape(im_padded))

padded image size (18600, 26600, 4)


In [51]:
subset_idx = 0
dict_idx = {'idx':[], 'x0':[], 'y0':[]}
for y in np.arange(subset_overlap, im_pad_y, subset_sz):
    for x in np.arange(subset_overlap, im_pad_x, subset_sz):
        #extract subset
        subset_y0 = y - subset_overlap
        subset_x0 = x - subset_overlap
        subset_yn = y + subset_sz + subset_overlap
        subset_xn = x + subset_sz + subset_overlap
        subset_im = im_padded[subset_y0:subset_yn,subset_x0:subset_xn,0:3]
        #write to file
        out_ffn = '/'.join([output_path, str(subset_idx).zfill(4)]) + '.png'
        im = Image.fromarray(subset_im)
        im.save(out_ffn)
        #append to dict
        dict_idx['idx'].append(subset_idx)
        dict_idx['y0'].append(subset_y0)
        dict_idx['x0'].append(subset_x0)
        #print('generated output for',out_ffn)
        subset_idx += 1
print('finished')

#write dictionary to file
df = pd.DataFrame.from_dict(dict_idx)
df.to_csv(output_ffn)

finished
